In [1]:
NPROCS=14

hamiltonian_encodings= [(0, 1, 2, 4, 0, 1, 3, 5), (0, 1, 2, 4,  1, 2, 3, 5), (0, 1, 2, 4, 1, 3, 4, 5), (0, 2, 3, 4, 1, 2, 3, 5), (0, 2, 3, 4, 1, 3, 4, 5), (0, 2, 4, 5, 1, 3, 4, 5), (0, 1, 2, 4, 0, 2, 3, 5), (0, 1, 2, 4, 0, 3, 4, 5), (0, 1, 2, 4, 2, 3, 4, 5), (0, 2, 3, 4, 0, 2, 3, 5), (0, 2, 4, 5, 0, 2, 3, 5), (0, 2, 3, 4, 0, 3, 4, 5), (0, 2, 4, 5, 0, 3, 4, 5), (0, 2, 3, 4, 2, 3, 4, 5), (0, 2, 4, 5, 2, 3, 4, 5), (1, 2, 3, 4, 0, 2, 3, 5), (1, 2, 4, 5, 0, 3, 4, 5), (1, 2, 3, 4, 0, 3, 4, 5), (1, 2, 3, 4, 2, 3, 4, 5), (1, 2, 4, 5, 2, 3, 4, 5), (2, 3, 4, 5, 2, 3, 4, 5)]

from gcaops.graph.formality_graph import FormalityGraph
def encoding_to_graph(encoding):
    targets = [encoding[0:4], encoding[4:8]]
    edges = sum([[(k+1,v) for v in t] for (k,t) in enumerate(targets)], [])
    return FormalityGraph(0, 6, edges)

hamiltonian_graphs = [encoding_to_graph(e) for e in hamiltonian_encodings]
print("Number of 4D Hamiltonians:", len(hamiltonian_graphs), flush=True)


from gcaops.algebra.differential_polynomial_ring import DifferentialPolynomialRing
from gcaops.algebra.superfunction_algebra import SuperfunctionAlgebra

D4 = DifferentialPolynomialRing(QQ, ('rho','a1','a2'), ('x','y','z','w'), max_differential_orders=[3+1,1+3+1,1+3+1])
rho, a1, a2 = D4.fibre_variables()
x,y,z,w= D4.base_variables()
even_coords = [x,y,z,w]

S4.<xi0,xi1,xi2,xi3> = SuperfunctionAlgebra(D4, D4.base_variables())
xi = S4.gens()
odd_coords = xi
epsilon = xi[0]*xi[1]*xi[2]*xi[3]


import itertools
from multiprocessing import Pool

def evaluate_graph(g):
    E = x*xi[0] + y*xi[1] + z*xi[2] + w*xi[3]
    result = S4.zero()
    for index_choice in itertools.product(itertools.permutations(range(4)), repeat=2):
        sign = epsilon[index_choice[0]] * epsilon[index_choice[1]]
        vertex_content = [S4(rho), S4(rho), S4(a1), S4(a1), S4(a2), S4(a2)]
        for ((source, target), index) in zip(g.edges(), sum(map(list, index_choice), [])):
            vertex_content[target] = vertex_content[target].derivative(even_coords[index])
        result += sign * prod(vertex_content)
    return result

print("Calculating 4D hamiltonian_formulas", flush=True)
hamiltonian_formulas = []
with Pool(processes=NPROCS) as pool:
    hamiltonian_formulas = list(pool.imap(evaluate_graph, hamiltonian_graphs))
print("Calculated 4D hamiltonian_formulas", flush=True)

hamiltonian_formulas.count(0)
[k for (k, ham) in enumerate(hamiltonian_formulas) if ham == 0]

from itertools import combinations
for i,j in combinations(range(len(hamiltonian_formulas)),2):
    if i!=j and hamiltonian_formulas[i]==hamiltonian_formulas[j]:
        print("4D Hamiltonians", i,j, "share the same formula")

for i,j in combinations(range(len(hamiltonian_formulas)),2):
    if i!=j and hamiltonian_formulas[i]==-hamiltonian_formulas[j]:
        print("4D Hamiltonians", i,j, "share the same formula with a - sign")

Number of 4D Hamiltonians: 21
Calculating 4D hamiltonian_formulas
Calculated 4D hamiltonian_formulas
4D Hamiltonians 3 15 share the same formula
4D Hamiltonians 4 17 share the same formula
4D Hamiltonians 5 16 share the same formula
4D Hamiltonians 10 11 share the same formula
4D Hamiltonians 13 18 share the same formula
4D Hamiltonians 14 19 share the same formula
4D Hamiltonians 1 6 share the same formula with a - sign
4D Hamiltonians 2 7 share the same formula with a - sign
